# Obliczanie wartości sinusa w dziedzinie liczb zespolonych

### Podstawowe funkcje i parametry

In [34]:
Prec = BigFloat
Comp = Tuple{Prec, Prec}

SIN_MAX_ITER = 10
COS_MAX_ITER = 10
SINH_MAX_ITER = 50
COSH_MAX_ITER = 50

setprecision = 128

# Mnożenie liczb zespolonych
function multiply_complex(x::Comp, y::Comp)
    return x[1] * y[1] - x[2] * y[2], x[1] * y[2] + x[2] * y[1]
end

# Kwadrat liczby zespolonej
function square_complex(x::Comp)
    return multiply_complex(x, x)
end

# Funkcja obliczająca n-tą potęgę liczby zespolonej,
# korzystając z algorytmu szybkiego potęgowania.
# Ogranicza do minimum wykonywaną liczbę mnożeń (w tym błąd).
function power_complex(x::Comp, n::Int64)
    if n > 0
        if n & 1 == 1
            y = square_complex(power_complex(x, n>>=1))
            return multiply_complex(y, x)
        else
            square_complex(power_complex(x, n>>=1))
        end
    else
        return (Prec(1), Prec(0))
    end
end

# Funkcja obliczająca wartość symbolu newtona
function newton(n::Int64, k::Int64)
    if n < k return 0 end
    if k < 0 return 0 end
    Σ = Prec(1)
    for i = 1:(n-k)
         Σ = Σ * (i + k) / i
    end
    return Σ
end


newton (generic function with 1 method)

## Przykład 1
Podstawowym pomysłem jest obliczanie sinusa z szeregu Taylora, kolejno obliczając potęgi i sumując liczby zespolone

In [1]:
function sin_complex_1(z)
    x = z[1]
    y = z[2]
    as = a = Prec(1)
    bs = b = Prec(x)
    cs = c = Prec(1)
    ds = d = Prec(y)
    for i = 1:MAX_SIN_ITER
        a = - a * prec(x) / prec(2  * i - 1) * prec(x) / prec(2 * i)
        b = - b * prec(x) / prec(2 * i) * prec(x) / prec(2 * i + 1)
        c = c * prec(y) / prec(2  * i - 1) * prec(y) / prec(2 * i)
        d = d * prec(y) / prec(2 * i) * prec(y) / prec(2 * i + 1)
        as += a
        bs += b
        cs += c
        ds += d
        @printf "%.8f, %.8f\n" cs * bs as * ds
    end
    return cs * bs, as * ds
end

@show sin_1(12, -6, 30, Float32)
print("\n")
@show sin_1(0.3, -6, 30, Float32)

-5244.00000000, 2982.00000000
131224.81250000, -84692.40625000
-731977.87500000, 544530.50000000
1598440.25000000, -1389740.87500000
-1903705.50000000, 1942775.75000000
1500016.00000000, -1783762.12500000
-868084.00000000, 1183001.37500000
389433.75000000, -599109.87500000
-140302.95312500, 239667.98437500
41331.88671875, -78190.48437500
-10358.96191406, 20882.45898438
2046.84875488, -4962.65917969
-497.93283081, 763.02874756
-46.64151382, -327.57843018
-116.51888275, -147.06414795
-106.99015045, -173.26783752
-108.14321136, -169.90480042
-108.01855469, -170.28915405
-108.03066254, -170.24978638
-108.02960968, -170.25341797
-108.02969360, -170.25312805
-108.02967834, -170.25314331
-108.02967834, -170.25314331
-108.02967834, -170.25314331
-108.02967834, -170.25314331
-108.02967834, -170.25314331
-108.02967834, -170.25314331
-108.02967834, -170.25314331
-108.02967834, -170.25314331
-108.02967834, -170.25314331
sin_1(12, -6, 30, Float32) = (-108.02968f0, -170.25314f0)

5.61450005, -40.110

(59.611057f0, -192.70393f0)

In [21]:


function square(x)
    return x * x
end

# Funkcja obliczająca szybko kolejne potęgi. Zachowuje precyzję x (typ x = typ wartości zwracanej)
function power(x, n)
    if n > 0
        if n & 1 == 1
            return square(power(x, n>>=1)) * x
        else 
            return square(power(x, n>>=1))
        end
    else 
        return 1
    end
end

# Funkcja obliczająca sinusa z liczby rzeczywistej x korzystając
# z rozwinięcia w szereg Taylora długości n
function sin_real(x::Prec)
    δx = x
    Σ = x
    for i = 1:SIN_MAX_ITER
        δx = - δx * x / (2 * i) * x / (2 * i + 1)
        Σ += δx
    end
    return Σ
end

# Funkcja obliczająca cosinusa z liczby rzeczywistej x korzystając
# z rozwinięcia w szereg Taylora długości n
function cos_real(x::Prec)
    δx = Prec(1)
    Σ = Prec(1)
    for i = 1:COS_MAX_ITER
        δx = Prec(- δx  * x / (2i - 1) * x / (2i))
        Σ += δx
    end
    return Σ
end

function sin_nx(nx, n)
    Σ = Prec(0)
    x = Prec(nx) / prec(n)
    cos = cos_real(x)
    sin = sin_real(x)
    for i = 0:div(n, 2)
        Σ += (i % 2 == 0 ? 1 : -1) * newton(n, 2 * i + 1) * power(cos, n - 2 * i - 1) * power(sin, 2 * i + 1)
    end
    return Σ
end

function cos_nx(nx, n)
    Σ = Prec(0)
    x = Prec(nx) / Prec(n)
    cos = cos_real(x)
    sin = sin_real(x)
    for i = 0:div(n, 2)
        Σ += (i % 2 == 0 ? 1 : -1) * newton(n, 2 * i) * power(cos, n - 2 * i) * power(sin, 2 * i)
    end
    return Σ
end


cos_2x (generic function with 2 methods)

<h2>Test #1</h2>
Sprawdzamy, dla jakiego n skracanie przedziału \[0, &pi;/2) korzystając ze wzoru na sinus zwielokrotnionego kąta ma sens.

In [7]:
tests = [Float32(0.321563716), Float32(0.034321248), Float32(1.0523452), Float32(0.00000123456), Float32(1.000003123)]
range = 10000
max_n = 30
Prec = BigFloat


for k = 1:max_n
    Σ² = 0
    for i = 1:5
    curr_test = tests[i]
        for j = 1:range
            curr_test = nextfloat(curr_test)
            target = sin(Float64(curr_test))
            Δ = target - sin_nx(curr_test, k, 10)
            Σ² += Δ * Δ
        end
    end
    @printf "n = %d, σ = %e \n" k sqrt(Σ² / range)
end


n = 1, σ = 5.964896e-08 
n = 2, σ = 7.611648e-08 
n = 3, σ = 1.081264e-07 
n = 4, σ = 1.522711e-07 
n = 5, σ = 1.951855e-07 
n = 6, σ = 2.494462e-07 
n = 7, σ = 2.345645e-07 
n = 8, σ = 2.444420e-07 
n = 9, σ = 2.608310e-07 
n = 10, σ = 2.547575e-07 
n = 11, σ = 3.373939e-07 
n = 12, σ = 4.134634e-07 
n = 13, σ = 4.050933e-07 
n = 14, σ = 4.473660e-07 
n = 15, σ = 3.603979e-07 
n = 16, σ = 4.290815e-07 
n = 17, σ = 6.139407e-07 
n = 18, σ = 5.435317e-07 
n = 19, σ = 7.554794e-07 
n = 20, σ = 7.605540e-07 
n = 21, σ = 7.416079e-07 
n = 22, σ = 6.872961e-07 
n = 23, σ = 7.825146e-07 
n = 24, σ = 8.439078e-07 
n = 25, σ = 9.062228e-07 
n = 26, σ = 7.096421e-07 
n = 27, σ = 8.207468e-07 
n = 28, σ = 9.412922e-07 
n = 29, σ = 6.661806e-07 
n = 30, σ = 6.732113e-07 


In [4]:
k = 100
n = k / 2
x = Float64(1.0)
while x > 0.5
    x *= k * k / (2n - 1) / (2n)
    n += 1
end

@show n

n = 57.0


57.0

In [31]:
exact_sinh = Prec("10.0178749274099018989745936194658280601781041231828634644056532510463926051808870905252214580081921788136031436005276604654731845463086661965545666809253301856979834214884134800415226711700580868934289537240194339117071445876738421988703293310666662074873025393922419074202969904427384978934547999669783367996478777737387515158099245608102468799707910743158654767792319545286680090845441765466693103005564580755152769830821020898523711510472417274773015585649780529003697102125038131242050195636504635373129836335784769819562604683637035572605572218826029004693336598037776759038456992068734019962315072590278266626183660525078196734326386033216758620007131926640350061262077283570845419590562070693093134140875714806032858928269295308563453677379062041720184214492393331096756140382287179704817388080282537490855424255503455845715185625683972134857957278057234162657813792323741991")

function sinh_real(x::Prec)
    Σ = x
    δx = x
    for i = 1:SINH_MAX_ITER
        δx = δx * x / (2 * i) * x / (2 * i + 1)
        Σ += δx
        @show Σ - exact_sinh
    end
    return Σ
end

function cosh_real(x::Prec)
    Σ = Prec(1)
    δx = Prec(1)
    for i = 1:COSH_MAX_ITER
        δx = δx * x / (2 * i - 1) * x / (2 * i)
        Σ += δx
        @show Σ
    end
    return Σ
end

function sin_complex_n(z, n)
    Σ_r = Prec(0)
    Σ_i = Prec(0)
    cos = cos_complex(z)
    sin = sin_complex(z)
    for i = 0:div(n, 2)
        σ = multiply_complex(power_complex(cos, n - 2 * i - 1), power_complex(sin, 2 * i + 1))
        Σ_r += (i % 2 == 0 ? 1 : -1) * newton(n, 2 * i + 1) * σ[1]
        Σ_i += (i % 2 == 0 ? 1 : -1) * newton(n, 2 * i + 1) * σ[2]
    end
    return Σ_r, Σ_i
end


function sin_complex(z)
    return sin_real(z[1]) * cosh_real(z[2]), cos_real(z[1]) * sinh_real(z[2])
end

z = Prec(1), Prec(3)
exact_result = Prec("8.471645454300149424897583639976881207279850599273339864612249252167060156811179"), 
            Prec("5.41268092317819278427941063379302764217781251017215863262723484500606267062513")
z = sin_complex(z)
@show z[1] - exact_result[1]
@show z[2] - exact_result[2]

@show exact_sinh - nextfloat(exact_sinh)

Σ = 5.500000000000000000000000000000000000000000000000000000000000000000000000000000
Σ = 8.875000000000000000000000000000000000000000000000000000000000000000000000000000
Σ = 9.887499999999999999999999999999999999999999999999999999999999999999999999999972
Σ = 1.005022321428571428571428571428571428571428571428571428571428571428571428571433e+01
Σ = 1.00664955357142857142857142857142857142857142857142857142857142857142857142858e+01
Σ = 1.006760501217532467532467532467532467532467532467532467532467532467532467532483e+01
Σ = 1.006765987639592550306836021121735407449693163978878264592550306836021121735427e+01
Σ = 1.006766193380419803410874839446268017696589125160553731982303410874839446268039e+01
Σ = 1.006766199431620604972758334102871917997968418136485363376119678640687044048416e+01
Σ = 1.006766199574938518693960837923686220899843190864862691488078484982509750311635e+01
Σ = 1.006766199577730426104114133452663122904425166956973938139610150041116686147936e+01
Σ = 1.0067661995777759463336275024

-1.381786968815111140061816298048063931378560058309805021603792555226974688505988e-76